In [ ]:
# Cell 1: Installs
!pip install -q -U transformers accelerate bitsandbytes torch unsloth
!pip install -q gspread google-auth google-auth-oauthlib google-auth-httplib2 pandas streamlit pyngrok
# Kill the current Colab runtime kernel after installs to use updated packages
import os
os.kill(os.getpid(), 9)

ERROR: Operation cancelled by user
^C


In [16]:

# import secrets from .env
import os
import dotenv
# Load environment variables from .env file
dotenv_path = '.env'
if os.path.exists(dotenv_path):
    dotenv.load_dotenv(dotenv_path)
    print("Environment variables loaded from .env file.")
else:
    print("No .env file found. Make sure to create one with your secrets.")


Environment variables loaded from .env file.


In [ ]:
# Cell 2: Imports
from unsloth import FastLanguageModel
import os
import gspread

from google.oauth2 import service_account
import google.auth
import pandas as pd
import torch
# Remove or comment out direct transformers model/tokenizer if Unsloth handles both
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoTokenizer # Keep Tokenizer for now, Unsloth might load it too

import json
import logging
import time
import re
from io import StringIO
import traceback # For detailed error logging

# Optional: Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_15777/177454013.py:13: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel # Import Unsloth's loader


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compiler.py:496: SyntaxWarning: invalid escape sequence '\s'
  left = re.match("[\s\n]{4,}", leftover).span()[1]
/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compiler.py:924: SyntaxWarning: invalid escape sequence '\*'
  .replace("*", "\*").replace("^", "\^")\
/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compiler.py:924: SyntaxWarning: invalid escape sequence '\^'
  .replace("*", "\*").replace("^", "\^")\
/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compiler.py:925: SyntaxWarning: invalid escape sequence '\-'
  .replace("-", "\-").replace("_", "\_")\
/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compiler.py:925: SyntaxWarning: invalid escape sequence '\_'
  .replace("-", "\-").replace("_", "\_")\
/home/phuoc/git/AI_agent_gemma3/venv/lib/python3.12/site-packages/unsloth_zoo/compile

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!


In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from fastembed import TextEmbedding
import streamlit as st
# authorize autotokennizer with HF token to download the model and tokenizer

from huggingface_hub import login
# Login to Hugging Face
HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
gemma_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it", 
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN
)

In [ ]:

@st.cache_resource
def load_models():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    # Load the Gemma model and tokenizer
    gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it",token=HF_TOKEN) 
    gemma_model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-3-4b-it", 
        quantization_config=bnb_config,
        device_map="auto",
        token=HF_TOKEN
    )
    # Load the SQL model and tokenizer
    sql_tokenizer = AutoTokenizer.from_pretrained("PipableAI/pip-sql-1.3b")
    sql_model = AutoModelForCausalLM.from_pretrained(
        "PipableAI/pip-sql-1.3b",
        quantization_config=bnb_config,
        device_map="auto",
        token=HF_TOKEN
    )
    embedding_model = TextEmbedding(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
    return gemma_tokenizer, gemma_model, sql_tokenizer, sql_model, embedding_model

gemma_tokenizer, gemma_model, sql_tokenizer, sql_model, embed_model = load_models()

In [ ]:
import sqlite3
from qdrant_client import QdrantClient, models

DB_PATH = "/content/drive/MyDrive/YourApp/mydatabase.db" # Make sure this dir exists
QDRANT_COLLECTION_PREFIX = "table_"

@st.cache_resource
def init_qdrant():
    client = QdrantClient(":memory:")
    return client

@st.cache_resource # Or manage connection state differently if schema changes often
def connect_db(path):
     conn = sqlite3.connect(path, check_same_thread=False) # check_same_thread=False needed for Streamlit
     return conn

qdrant_client = init_qdrant()
db_conn = connect_db(DB_PATH)